<a href="https://colab.research.google.com/github/w1756015-m-zidani/Final-Year-Project/blob/main/Final_verson_tweet_cleaning_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vaderSentiment

In [ ]:
pip install afinn

Import Librarys

In [ ]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from afinn import Afinn
import spacy


# Load the spaCy language model
nlp = spacy.load('en_core_web_sm')


Define Functions for Tokenization and Cleaning of Tweets

In [ ]:

# Define a function to perform tokenization and lemmatization using spaCy
def tokenize_and_lemmatize(text):
    # Tokenize the text using spaCy
    doc = nlp(text)
    # Lemmatize each token and remove punctuation, stop words, and whitespace
    tokens = [token.lemma_.lower().strip() for token in doc if not token.is_punct and not token.is_stop and not token.is_space]
    # Join the tokens back into a string
    return ' '.join(tokens)

# Define a function to clean the tweets
def clean_tweet(tweet):
    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    # Remove mentions and hashtags
    tweet = re.sub(r'@\w+|#\w+', '', tweet)
    # Remove punctuation
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remove non-alphanumeric characters
    tweet = re.sub(r'[^A-Za-z0-9\s]+', '', tweet)
    # Convert everything to lowercase
    tweet = tweet.lower()
    # Tokenize and lemmatize the tweet
    tweet = tokenize_and_lemmatize(tweet)


    # Remove stop words #### I dont think i need this because its already done ####
   # stop_words = set(stopwords.words('english'))
   # words = nltk.word_tokenize(tweet)
   # words = [word for word in words if not word in stop_words]
   # tweet = ' '.join(words)
    return tweet


Load & clean the Data

In [ ]:
# Load the dataset
df = pd.read_csv('Tweets Final.csv', encoding='ISO-8859-1')

# Clean the tweets
df['cleaned_text'] = df['Tweet'].apply(clean_tweet)


In [ ]:
df.head()


Remove non-english tweets

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect

In [ ]:

# Define a function to remove non-English tweets
def remove_non_english_tweets(df):
    df['lang'] = df['Tweet'].apply(lambda x: detect(x))
    df = df[df['lang']=='en']
    df = df.drop(['lang'], axis=1)
    return df

In [ ]:
df2 = remove_non_english_tweets(df)

In [ ]:
df2.shape

(147396, 2)

In [ ]:
df2.head()

,Tweet,cleaned_text
0,The Daily Mail has really turned on Boris John...,daily mail turn boris johnson guess owner edit...
2,More Tax payers money Laundering through Boris...,tax payer money laundering boris johnson bill ...
3,@paulwaugh We love Boris Johnson \n\nThe Briti...,love boris johnson british people place trust ...
4,Just like the time he reframed the google resu...,like time reframe google result boris johnson ...
6,@44tunafish @northumbrian_ @BootsVernon @Jim_C...,like boris johnson explicitly not want people ...


Perform Sentiment Analysis using TextBlob

In [ ]:
# Perform sentiment analysis using TextBlob
df2['polarity_textblob'] = df2['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.polarity)


Perform Sentiment Analysis using VADER

In [ ]:
# Perform sentiment analysis using VADER
analyzer = SentimentIntensityAnalyzer()
df2['sentiment_vader'] = df2['cleaned_text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])


Perform Sentiment Analysis using AFINN

In [ ]:
# Perform sentiment analysis using AFINN
afinn = Afinn()
df2['sentiment_afinn'] = df2['cleaned_text'].apply(lambda x: afinn.score(x))


Print Result

In [ ]:
# Print the results
df2.head()

In [ ]:
import pandas as pd
df2.to_excel('testoutputdeleteafter.xlsx', index=False)

Calculating propotion of sentiment

In [ ]:
def calculate_sentiment_proportions(df, sentiment_column):
    # Get the number of positive, negative, and neutral tweets
    num_positive = len(df[df[sentiment_column] > 0])
    num_negative = len(df[df[sentiment_column] < 0])
    num_neutral = len(df[df[sentiment_column] == 0])
    # Calculate the proportions of positive, negative, and neutral tweets
    total = num_positive + num_negative + num_neutral
    prop_positive = (num_positive / total)*100
    prop_negative = (num_negative / total)*100
    prop_neutral = (num_neutral / total)*100
    # Print Output

    print("Postive", prop_positive)
    print("Negative", prop_negative)
    print("Neutral", prop_neutral)


In [ ]:
#Textblob
print("Textblob:")
calculate_sentiment_proportions(df2,'polarity_textblob')
print()
#VADAR
print("VADAR:")
calculate_sentiment_proportions(df2,'sentiment_vader')
print()
print("AFINN")
#AFINN
calculate_sentiment_proportions(df2,'sentiment_afinn')
